In [1]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from tqdm import tqdm


from models import FlexiAST_V2
import numpy as np
import dataloader
from utilities import *


model = FlexiAST_V2(
    sample_size = (128,3072),
    patch_size = 16,
    num_classes = 44,
    dim = 768,
    depth = 12,
    heads = 12,
    dropout = 0,
    emb_dropout = 0,
    token_dropout_prob = 0,  # token dropout of 10% (keep 90% of tokens)
    channels=1,
)


model_load = "/mnt/bear1/users/fj/FlexiAST_V2/flexiast2/epic-sound/epic-sound_flexi_len_pretrained-max3072-f48-t192-offcial-scheduler-1e-5/models/audio_model.12.pth"


out_dict = {}
state_dict = torch.load(model_load,map_location=torch.device('cpu'))
for k, v in state_dict.items(): # Adjust the name of dict
    out_dict[k[7:]] = v
model.load_state_dict(out_dict, strict=True)

/home/jfeng/anaconda3/envs/flexiast2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



<All keys matched successfully>

In [2]:
import copy
from fvcore.common.config import CfgNode
import yaml
from epic_data import loader

# Load the YAML file
yaml_file = "/home/jfeng/FJ/epic-sounds-annotations/src/config.yaml"
with open(yaml_file, 'r') as f:
    cfg_dict = yaml.safe_load(f)


length = np.arange(128*2, 128*24+1,128)
# append two special cases
length = np.append(length, 400)
length = np.append(length, 1600)
length = np.sort(length)

quality_acc = []

ori_size = get_shape(16,16,16,input_tdim=int(1024))


for l in length:
    # Convert dictionary to CfgNode
    cfg = CfgNode(cfg_dict)
    cfg.TRAIN.DATASET = "Epicsounds_flexi"
    cfg.T_MASK = 0
    cfg.F_MASK = 0
    cfg.T_WARP = 5
    cfg.AUDIO_DATA.NUM_FRAMES = int(l)
    # cfg.MIN_AUDIO_LENGTH = int(128*q)
    cfg.MIN_AUDIO_LENGTH = 192
    cfg.DATA_LOADER.NUM_WORKERS=16
    cfg.RANDOM=False


    val_loader = loader.construct_loader_flexi(cfg, "val")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        A_predictions = []
        A_targets = []
        A_loss = []
        with tqdm(val_loader) as vepoch:
            for data_items, labels in vepoch:
                data_items, labels = data_items, labels
                for i in range(len(data_items)):
                    data_items[i] = data_items[i].to(device)
                outputs = model(data_items,group_samples=True,group_max_seq_len=2048)
                outputs = torch.sigmoid(outputs)
                predictions = outputs.to('cpu').detach()

                A_predictions.append(predictions)
                A_targets.append(labels)

                labels = labels.to(device)


        outputs = torch.cat(A_predictions)
        labels = torch.cat(A_targets)
        stats = calculate_stats(outputs, labels)

    mAP = np.mean([stat['AP'] for stat in stats])
    acc = stats[0]['acc']
    print(f"Quality ratio: {l}, mAP: {mAP}, acc: {acc}")
    quality_acc.append(acc)

print(quality_acc)


100%|██████████| 670/670 [00:59<00:00, 11.19it/s]


Quality ratio: 256, mAP: 0.19271589585240695, acc: 0.48164281269446174


100%|██████████| 670/670 [01:00<00:00, 11.05it/s]


Quality ratio: 384, mAP: 0.2009552937633239, acc: 0.5001866832607342


100%|██████████| 670/670 [01:00<00:00, 11.12it/s]


...

Quality ratio: 2816, mAP: 0.2185663668365763, acc: 0.5230864965774735


100%|██████████| 670/670 [01:02<00:00, 10.78it/s]


Quality ratio: 2944, mAP: 0.2203509604447045, acc: 0.5225886745488488


100%|██████████| 670/670 [01:03<00:00, 10.49it/s]


Quality ratio: 3072, mAP: 0.22008337532639055, acc: 0.5224642190416926
[0.48164281269446174, 0.5001866832607342, 0.5001866832607342, 0.5072806471686372, 0.5108898568761667, 0.5133789670192906, 0.5157436216552582, 0.5176104542626011, 0.5176104542626011, 0.5184816428126945, 0.5189794648413192, 0.5196017423771002, 0.5203484754200374, 0.5212196639701306, 0.5222153080273803, 0.5218419415059117, 0.5218419415059117, 0.5224642190416926, 0.523335407591786, 0.5240821406347231, 0.5229620410703174, 0.5228375855631612, 0.5230864965774735, 0.5225886745488488, 0.5224642190416926]


In [ ]:
baseline_1024 = [0.3684401891, 0.5022421525, 0.5124227366, 0.5649012241, 0.587686341, 0.5941098049, 0.5928978306, 0.5928978306, 0.5901102897, 0.5811416798, 0.5721730699, 0.5515695067, 0.5413889226, 0.5307235487, 0.5041813114, 0.4749727306, 0.4347351836, 0.4257665737, 0.3977699673, 0.3736516786, 0.3490486002, 0.3278390498, 0.3069930917, 0.2876015028, 0.2848139619]
baseline_400  = [0.4262513635, 0.4805478124, 0.4897588171, 0.4822445764, 0.4674584899, 0.434856381, 0.3931644649, 0.350139377, 0.3050539329, 0.2694218883, 0.2363349897, 0.209671555, 0.193915889, 0.1889467943, 0.1708883772, 0.1554963035, 0.1426493758, 0.1334383711, 0.1190158769, 0.1087140953, 0.1016846443, 0.09598836505, 0.09247363956, 0.08386862199, 0.08144467337]
baseline_1600 = [0.3245667192, 0.4637013695, 0.4747303357, 0.5379953945, 0.5703551085, 0.5828384438, 0.5936250151, 0.5935038177, 0.5943521997, 0.5953217792, 0.5948369895, 0.5932614229, 0.5966549509, 0.5937462126, 0.5862319719, 0.5778693492, 0.5739910314, 0.5682947522, 0.5532662708, 0.542358502, 0.525754454, 0.5086656163, 0.4984850321, 0.4822445764, 0.4679432796]
len_cut_flexi = [0.44661253181432553, 0.5235729002545146, 0.5607805114531572, 0.5715670827778451, 0.580050902920858, 0.588534723063871, 0.5893831050781724, 0.5924130408435341, 0.5935038177190644, 0.5944733971639802, 0.5947157920252091, 0.5943521997333656, 0.5945945945945946, 0.5943521997333656, 0.5943521997333656, 0.5943521997333656, 0.5939886074415223, 0.5943521997333656, 0.5943521997333656, 0.5941098048721367, 0.5939886074415223, 0.5939886074415223, 0.5939886074415223]

In [ ]:
# draw the plot
import matplotlib.pyplot as plt
plt.plot(length, baseline_400, label="Baseline-400",linestyle='--')
plt.plot(length, baseline_1024, label="Baseline-1024",linestyle=':')
plt.plot(length, baseline_1600, label="Baseline-1600",linestyle='-')
plt.plot(length, len_cut_flexi, label="FlexiV2-cut",color='r',linewidth=2)
# plt.axhline(y=len_cut_natural, color='r', linestyle='-', label="FlexiV2-Natural", linewidth=2)
# plt.text(0, len_cut_natural, f"          {len_cut_natural}", verticalalignment='bottom', color='r')
# plt.axhline(y=len_cut_natural, color='r', linestyle='-', label="FlexiV2-Natural")
# plt.axhline(y=len_cut_natural, color='r', linestyle='-', label="FlexiV2-Natural", linewidth=2)
# plt.text(0, len_cut_natural, f"          {len_cut_natural}", verticalalignment='bottom', color='r')
plt.legend()
plt.xlabel("Time Dimension")
plt.ylabel("Accuracy")
plt.title("Quality ratio vs Accuracy")
# show grid


# show a vertical line
# plt.grid(True)
# set x-axis indicator
# plt.xticks(range(128, 3072, 256))
# plt.axvline(x=475, color='r', linestyle='--')
# plt.axvline(x=1124, color='r', linestyle='--')


# set y-axis range
plt.ylim(0.0, 0.75)

plt.show()